# **Lab 4: Multi-Layer Perceptron**
**Name(s):** Luke Voinov, Tiffany Nguyen, Emmanuel Garcia, Nimai Keshu

We use the US Census dataset:

https://www.kaggle.com/muonneutrino/us-census-demographic-data/data

##### **Classification Task:**

The classification task you will be performing is to predict, **for each tract, what the child poverty rate will be**. You will need to convert this from regression to four levels of classification by quantizing the variable of interest.

In [ ]:

# Import any dependencies

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import expit


### **1. Load, Split, and Balance**

*1.1. Load the data into memory and save it to a pandas data frame. Do not normalize or one-hot encode any of the features until asked to do so later in the rubric.*

In [ ]:
path = "/content/acs2017_census_tract_data.csv"
df = pd.read_csv(path)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/acs2017_census_tract_data.csv'

In [ ]:
df.info()

*1.2 Remove any observations that having missing data.*

In [ ]:
"""
Visualize missing data
"""

import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

# External package: conda install missingno
import missingno as mn

mn.matrix(df)
plt.title("Missing Data Visual",fontsize=22)
plt.show()

In [ ]:
print(df.isnull().sum())  # Shows the count of missing values per column

We can see that no column is missing too much data. Therefore, it is okay to leave these missing values as they are; imputation is not necessary.

In [ ]:
"""
Remove all rows that have a population of less than 100 people.
Delete any rows that contain missing data
"""
# keep rows with TotalPop >= 100, reset index
df_mod = df[df['TotalPop'] >= 100].reset_index(drop=True)

# also delete any row that has any missing value
df_clean = df_mod.dropna().reset_index(drop=True)

In [ ]:
"""
Visualize missing data
"""

import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

# External package: conda install missingno
import missingno as mn

mn.matrix(df_clean)
plt.title("Missing Data Visual",fontsize=22)
plt.show()


df_clean.count().head()

We started with 74001 and ended with 72708 people. This means that deleting every row with a missing value deleted ~2% of the data. This is small and permits deletion.

*1.3 Encode any string data as integers for now.*

We have already seen that there are only two columns that contain strings: the state and the county

In [ ]:
# encode states
states = df_clean['State'].unique()
count = 0

df_state_enc = df_clean
for s in states:
    df_state_enc = df_state_enc.replace(s,count)
    count += 1

print("States encoded to ints:\n", df_state_enc['State'].unique())

# encode counties
counties = df_clean['County'].unique()
count = 0

df_enc = df_state_enc
for c in counties:
    df_enc = df_enc.replace(c,count)
    count += 1

print("Counties encoded to ints:\n", df_enc['County'].unique())


In [ ]:
print(df_enc.info(verbose=False)) # Notice the dtypes
df_enc.head()

1.4 You have the option of keeping the "county" variable or removing it. Be sure to discuss why you decided to keep/remove this variable.

We decided to keep 'county' because we suspect each county will meaningfully inform child poverty rates.

In [ ]:
# see how many instances there are for each county
counties = df_enc['County'].unique()
count = 0
num_each_instance = []

for c in counties:
    bools = df_enc['County'] == c
    num_each_instance.append(df_enc.where(bools == True)['County'].count())


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(num_each_instance,bins=200)
plt.show()

plt.figure()
plt.boxplot(num_each_instance)
plt.show()

print("Mean:", np.mean(num_each_instance))
print("Median:", np.median(num_each_instance))

Here, as we can see from the graphs, a median is a more representative value due to the tremendous effect of outliers. This median shows that each county has around 9 census tracts, and each census tract has around 1k people. While all the counties are in the same state with the same laws and policies, they can still vary dramatically between each other. As an anecdote, it would be reasonable to suspect that child poverty rates are lower in the Highland Park county as compared to the Oak Cliff county.

We conclude our decision with a PCA analysis that demonstrate how important of a feature counties are: **CAN I DO THIS**?

The next two requirements will need to be completed together as they might depend on one another.

*Note: You will need to one hot encode the target, but **do not** one hot encode the categorical data **until** instructed to do so in the lab.*

1.5 Balance the dataset so that about the same number of instances are within each class. Choose a method for balancing the dataset and explain your reasoning for selecting this method. One option is to choose quantization thresholds for the "ChildPoverty" variable that equally divide the data into four classes. *Should balancing of the dataset be done for both the training and testing set? Explain.*

We balanced the dataset according to 4 percentiles: 75% > in poverty level = impoverished, 75 - 50 % are poor, 50 - 25 % are well-off, 25 % < are rich. All of these results will be relative to the training data and will equally split 1/4 of the dataset into each class.

The training dataset must be balanced because it will allow us to categorize among the 4 classes. The test dataset should also be balanced to reflect the real-world use case. If it's unbalanced, then a split test case may result in 99% impoverished instances and 1% of the rest, which is inapproprite if we want to test real-life scenarios.





In [ ]:
# This code is adapted from github Copilot.
# It will split the dataset into 4 quartiles and place 25% of the dataset into a quartile for each quartile, thus evenly distributing the data.


q25 = df_enc['ChildPoverty'].quantile(0.25)
q50 = df_enc['ChildPoverty'].quantile(0.50)
q75 = df_enc['ChildPoverty'].quantile(0.75)

# Create balanced classes
def classify_poverty(value):
    if value <= q25:
        return 0  # rich
    elif value <= q50:
        return 1  # well-off
    elif value <= q75:
        return 2  # poor
    else:
        return 3  # impoverished

df_enc['PovertyClass'] = df_enc['ChildPoverty'].apply(classify_poverty)

In [ ]:
df_enc['PovertyClass']

1.6 Assume you are equally interested in the classification performance for each class in the dataset. Split the dataset into 80% for training and 20% for testing. There is no need to split the data multiple times for this lab.

In [ ]:

# This code is adapted from voinov_lab3

from sklearn.preprocessing import StandardScaler

# Drop irrelevent columns and classes from feature list
X = df_enc.drop(columns=["TractId", "ChildPoverty", "PovertyClass"])
y = df_enc["ChildPoverty"]

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Previously the quantile included all the data. Now it has to only include 80% of the data. The test quantiles will be the same as the train one to be realistic.
q25 = y_train.quantile(0.25)
q50 = y_train.quantile(0.50)
q75 = y_train.quantile(0.75)

# Now make the classes
y_train = y_train.apply(classify_poverty).values
y_test = y_test.apply(classify_poverty).values

# Convert to numpy arrays with proper data types. Provided by github copilot to fix type issues
X_train = X_train.astype(np.float32).values
X_test = X_test.astype(np.float32).values

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nTraining set shape:", X_train_scaled.shape)
print("Test set shape:", X_test_scaled.shape)

In [ ]:
# LOOK: this is from someone else's lab, works with Tiffany's code
# so we may need to do some revisions in terms of prepping the data

from collections import Counter

X = df_enc.drop(columns=["PovertyClass"]) # features
y = df_enc["PovertyClass"] # target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

counts_train = Counter(y_train)
min_size = min(counts_train.values())

df_train = pd.concat([X_train, y_train], axis=1)
df_train_balanced = (df_train.groupby('PovertyClass')).apply(lambda x: x.sample(min_size, random_state=42)).reset_index(drop=True)

X_train_balanced = df_train_balanced.drop(columns=["PovertyClass"])
y_train_balanced = df_train_balanced["PovertyClass"]

print("\nFinal Shapes:")
print("X_train_balanced:", X_train_balanced.shape)
print("y_train_balanced:", y_train_balanced.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

y_train = y_train_balanced
X_train = X_train_balanced

### **2. Pre-processing and Initial Modeling**

2.1  You will be using a two layer perceptron from class for the next few parts of the rubric. There are several versions of the two layer perceptron covered in class, with example code. When selecting an example two layer network from class be sure that you use: (1) vectorized gradient computation, (2) mini-batching, (3) cross entropy loss, and (4) proper Glorot initialization, at a minimum. There is no need to use momentum or learning rate reduction (assuming you choose a sufficiently small learning rate). **It is recommended to use sigmoids throughout the network, but not required**

2.2 Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Do not normalize or one-hot encode the feature data (not yet). Be sure that training converges by graphing the loss function versus the number of epochs.

This is based off of the Two Layer Perceptron from Eric Larson's version from the class. We modified it so that we would be able to use minibatches_size as one of the parameters and made it suitable for multi-class classification by using softmax. The mathematics behind is consistently the same as the one in class.

In [ ]:
# Imported from 07. MLP Neural Networks.ipynb (Eric Larson, CS 5324/7324)

# Example adapted from https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch12/ch12.ipynb
# Original Author: Sebastian Raschka

# This is the optional book we use in the course, excellent intuitions and straightforward programming examples
# please note, however, that this code has been manipulated to reflect our assumptions and notation.

class TwoLayerPerceptron:
    def __init__(self, n_hidden=30, C=0.0, epochs=500, eta=0.001, minibatch_size=64, random_state=None):
        np.random.seed(random_state)
        self.n_hidden = n_hidden
        self.l2_C = C
        self.epochs = epochs
        self.eta = eta
        self.minibatch_size = minibatch_size

    @staticmethod
    def _encode_labels(y):
        """Encode labels into one-hot representation."""
        return pd.get_dummies(y).values.T

    def _initialize_weights(self, n_features, n_output):
        """Initialize weights using Glorot initialization."""
        limit_1 = np.sqrt(6 / (n_features + self.n_hidden))
        limit_2 = np.sqrt(6 / (self.n_hidden + n_output))

        W1 = np.random.uniform(-limit_1, limit_1, (self.n_hidden, n_features + 1))  # +1 for bias term
        W2 = np.random.uniform(-limit_2, limit_2, (n_output, self.n_hidden + 1))  # +1 for bias term
        b1 = np.zeros((self.n_hidden, 1))
        b2 = np.zeros((n_output, 1))

        return W1, W2, b1, b2

    @staticmethod
    def _sigmoid(z):
        """Sigmoid activation function."""
        return expit(z)

    @staticmethod
    def _softmax(z):
        """Softmax activation function (for cross-entropy)."""
        exp_values = np.exp(z - np.max(z, axis=0, keepdims=True))  # stability
        return exp_values / np.sum(exp_values, axis=0, keepdims=True)

    @staticmethod
    def _cross_entropy_loss(Y_enc, A3):
        """Cross-entropy loss function."""
        m = Y_enc.shape[1]
        cost = -np.sum(Y_enc * np.log(A3 + 1e-8)) / m  # add small epsilon for stability
        return cost

    def _cost(self, A3, Y_enc, W1, W2):
        """Compute the cost function with regularization."""
        cost = self._cross_entropy_loss(Y_enc, A3)
        L2_term = self._L2_reg(self.l2_C, W1, W2)
        return cost + L2_term

    def _L2_reg(self, lambda_, W1, W2):
        """Compute L2 regularization cost."""
        return (lambda_ / 2.0) * (np.sum(W1[:, 1:] ** 2) + np.sum(W2[:, 1:] ** 2))

    def _feedforward(self, X, W1, W2, b1, b2):
        """Compute the feedforward step."""
        A1 = self._add_bias_unit(X)
        Z1 = W1 @ A1.T + b1
        A2 = self._sigmoid(Z1)
        A2 = self._add_bias_unit(A2.T)  # Add bias to hidden layer
        Z2 = W2 @ A2.T + b2
        A3 = self._softmax(Z2)  # Softmax for classification
        return A1, Z1, A2, Z2, A3

    def _add_bias_unit(self, X):
        """Add bias unit to the input data."""
        ones = np.ones((X.shape[0], 1))
        return np.hstack((ones, X))

    def _get_gradient(self, A1, A2, A3, Z1, Z2, Y_enc, W1, W2):
        """Compute gradient step using backpropagation."""
        V2 = A3 - Y_enc  # For cross-entropy loss

        # FIX: Transpose the result of matrix multiplication to match A2's shape
        # Original: V1 = A2 * (1 - A2) * (W2[:, 1:].T @ V2)
        delta = W2[:, 1:].T @ V2
        # A2 shape is (samples, hidden+1), delta shape is (hidden, samples)
        # We need to transpose delta to match A2's shape
        delta_t = delta.T  # Now shape is (samples, hidden)

        # We need to remove the bias column from A2 for element-wise multiplication
        A2_nobias = A2[:, 1:]  # Remove bias column, now shape is (samples, hidden)
        V1 = A2_nobias * (1 - A2_nobias) * delta_t

        # Now calculate gradients with correct shapes
        gradW2 = V2 @ A2
        gradW1 = V1.T @ A1
        gradb2 = np.sum(V2, axis=1).reshape((-1, 1))
        gradb1 = np.sum(V1.T, axis=1).reshape((-1, 1))

        # Regularize weights that are not bias terms
        gradW1[:, 1:] += self.l2_C * W1[:, 1:]
        gradW2[:, 1:] += self.l2_C * W2[:, 1:]

        return gradW1, gradW2, gradb1, gradb2

    def fit(self, X, y):
        """Train the model using mini-batches and update weights."""
        Y_enc = self._encode_labels(y)
        n_features = X.shape[1]
        n_output = Y_enc.shape[0]

        # Initialize weights
        W1, W2, b1, b2 = self._initialize_weights(n_features, n_output)

        cost_history = []
        for epoch in range(self.epochs):
            minibatch_cost = 0

            # Shuffle and split the data into mini-batches
            permutation = np.random.permutation(X.shape[0])
            X_shuffled = X[permutation]
            Y_enc_shuffled = Y_enc[:, permutation]

            for i in range(0, X.shape[0], self.minibatch_size):
                X_batch = X_shuffled[i:i+self.minibatch_size]
                Y_batch = Y_enc_shuffled[:, i:i+self.minibatch_size]

                # Feedforward
                A1, Z1, A2, Z2, A3 = self._feedforward(X_batch, W1, W2, b1, b2)

                # Compute cost
                minibatch_cost = self._cost(A3, Y_batch, W1, W2)

                # Backpropagation
                gradW1, gradW2, gradb1, gradb2 = self._get_gradient(A1, A2, A3, Z1, Z2, Y_batch, W1, W2)

                # Update weights and biases
                W1 -= self.eta * gradW1
                W2 -= self.eta * gradW2
                b1 -= self.eta * gradb1
                b2 -= self.eta * gradb2

            cost_history.append(minibatch_cost)

        self.W1, self.W2, self.b1, self.b2 = W1, W2, b1, b2
        self.cost_ = cost_history
        return self

    def predict(self, X):
        """Predict class labels."""
        _, _, _, _, A3 = self._feedforward(X, self.W1, self.W2, self.b1, self.b2)
        return np.argmax(A3, axis=0)

In [ ]:
# raw data
tlp_raw = TwoLayerPerceptron(n_hidden=30, epochs=100, eta=0.001, minibatch_size=64)
tlp_raw.fit(X_train.values, y_train.values)

# evaluate on test set
y_pred = tlp_raw.predict(X_test.values)
accuracy = accuracy_score(y_test, y_pred)

plt.plot(range(len(tlp_raw.cost_)), tlp_raw.cost_)
plt.xlabel("Epochs")
plt.ylabel("Cost (Loss)")
plt.title("Loss Function vs Epochs")
plt.tight_layout()
plt.show()

print(f"Accuracy: {accuracy}")

Based off of the graph above, the values vary between 1.35 and 1.42. The small learning rate and random mini-batch sampling might have likely produced the oscillations. I believe with further tuning with normalization and one-hot encoded is required to achieve stable results.

2.3 Now normalize the continuous numeric feature data

In [ ]:
# trying to normalize data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

# train and evaluate
tlp_norm = TwoLayerPerceptron(n_hidden=30, epochs=100, eta=0.001, minibatch_size=64)
tlp_norm.fit(X_train_norm, y_train)

# predict and evaluate
y_train_pred_norm = tlp_norm.predict(X_train_norm)
y_test_pred_norm = tlp_norm.predict(X_test_norm)

# calculate accuracy
train_acc_norm = accuracy_score(y_train, y_train_pred_norm)
test_acc_norm = accuracy_score(y_test, y_test_pred_norm)

print(f"Training Accuracy (Normalized): {train_acc_norm}")
print(f"Test Accuracy (Normalized): {test_acc_norm}")

plt.plot(range(len(tlp_norm.cost_)), tlp_norm.cost_)
plt.xlabel("Epochs")
plt.ylabel("Cost (Loss)")
plt.title("Loss Function vs Epochs (Normalized Data)")
plt.show()


With the data being normalized, we can see a strong downward trend, starting from 0.6 and dropping within the first 10-20 epochs. There's some loss that stabilizes near 0.05 after around 30 epochs. There aer some small fluctuations after convergence due to the mini-batch stochastic gradient descent, but this is normal. This is definitely an improvement from the previous model.

2.4 One hot encode the categorical feature data. Use the example two-layer perceptron network from the class example and quantify performance using accuracy. Be sure that training converges by graphing the loss function versus the number of epochs.  

In [ ]:
# one-hot encoding categorical features
X_combined = pd.concat([X_train, X_test])
X_combined_encoded = pd.get_dummies(X_combined)

# split back into train and test sets
X_train_onehot = X_combined_encoded.iloc[:len(X_train_balanced)]
X_test_onehot = X_combined_encoded.iloc[len(X_train_balanced):]

# normalize one-hot encoded data
scaler_onehot = StandardScaler()
X_train_onehot_norm = scaler_onehot.fit_transform(X_train_onehot)
X_test_onehot_norm = scaler_onehot.transform(X_test_onehot)

# training model with normalized + one hot encoded data
tlp_onehot = TwoLayerPerceptron(n_hidden=30, epochs=100, eta=0.001, minibatch_size=64)
tlp_onehot.fit(X_train_onehot_norm, y_train_balanced)

# evaluating the model
y_train_pred_onehot = tlp_onehot.predict(X_train_onehot_norm)
y_test_pred_onehot = tlp_onehot.predict(X_test_onehot_norm)

# calculating accuracies for both training and testing sets
train_acc_onehot = accuracy_score(y_train_balanced, y_train_pred_onehot)
test_acc_onehot = accuracy_score(y_test, y_test_pred_onehot)

print(f"Training Accuracy (One-Hot Encoded + Normalized): {train_acc_onehot}")
print(f"Test Accuracy (One-Hot Encoded + Normalized): {test_acc_onehot}")

plt.plot(range(len(tlp_onehot.cost_)), tlp_onehot.cost_)
plt.xlabel("Epochs")
plt.ylabel("Cost (Loss)")
plt.title("Loss Function vs Epochs (One-Hot Encoded + Normalized Data)")
plt.show()

The final model with normalized AND one-hot encoded data looks more or as stable as the previous model with only normalized data. The training loss shows a steep initial drop and stabilizes near zero after around 30 epochs, indicating that the model successfully converged and learned the relationships between the features efficiently. Based off of the accuracy scores, the final model is the most accurate compared to the rest.

2.4 Compare the performance of the three models you just trained. Are there any meaningful differences in performance? Explain, in your own words, why these models have (or do not have) different performances.

The preprocessing steps have a meaningful impact on model performance. The unnormalized model failed to converge, while normalization allowed the network to train successfully. Adding one-hot encoding further ensured that categorical variables were treated correctly, leading to efficient and stable learning. Overall, data preprocessing was critical for the perceptron’s convergence and accuracy.

In [ ]:
# adapted from Copilot to compare accuracies

acc_raw = accuracy_score(y_test, tlp_raw.predict(X_test))
acc_norm = accuracy_score(y_test, tlp_norm.predict(X_test))
acc_onehot = accuracy_score(y_test, tlp_onehot.predict(X_test))

print(f"Raw Data Accuracy: {acc_raw:.4f}")
print(f"Normalized Data Accuracy: {acc_norm:.4f}")
print(f"One-Hot + Normalized Accuracy: {acc_onehot:.4f}")

plt.bar(["Raw", "Normalized", "One-Hot+Norm"], [acc_raw, acc_norm, acc_onehot])
plt.ylabel("Accuracy")
plt.title("Model Performance Comparison")
plt.show()

*Use one-hot encoding and normalization on the dataset for the remainder of this lab assignment.*

The bar graph above compares the accuracy of the three models. The normalized and one-hot encoded + normalized models achieved slightly higher accuracies (~25.5%) than the raw data model (~24.5%). While the improvement is relatively small, it confirms that preprocessing, particularly normalization, which contributes to more stable and effective learning. The results suggest that proper feature scaling and encoding help optimize model performance, even if the gains appear marginal for this dataset.

### **3. Modeling**


3.1 Add support for a third layer in the multi-layer perceptron. Add support for saving (and plotting after training is completed) the average magnitude of the gradient for each layer, for each epoch (like we did in the flipped module for back propagation). For magnitude calculation, you are free to use either the average absolute values or the L1/L2 norm.
- Quantify the performance of the model and graph the magnitudes for each layer versus the number of epochs.

In [ ]:
class ThreeLayerPerceptron(object):
    def __init__(self, n_hidden1=30, n_hidden2=20, C=0.0, epochs=500, eta=0.001, minibatch_size=64, random_state=None):
        np.random.seed(random_state)
        self.n_hidden1 = n_hidden1
        self.n_hidden2 = n_hidden2
        self.l2_C = C
        self.epochs = epochs
        self.eta = eta
        self.minibatch_size = minibatch_size

    @staticmethod
    def _encode_labels(y):
        """Encode labels into one-hot representation."""
        return pd.get_dummies(y).values.T

    def initialize_weights(self, n_features, n_output):
        """Initialize weights using Glorot initialization."""
        limit_1 = np.sqrt(6 / (n_features + self.n_hidden1))
        limit_2 = np.sqrt(6 / (self.n_hidden1 + self.n_hidden2))
        limit_3 = np.sqrt(6 / (self.n_hidden2 + n_output))

        W1 = np.random.uniform(-limit_1, limit_1, (self.n_hidden1, n_features + 1))
        W2 = np.random.uniform(-limit_2, limit_2, (self.n_hidden2, self.n_hidden1 + 1))
        W3 = np.random.uniform(-limit_3, limit_3, (n_output, self.n_hidden2 + 1))

        b1 = np.zeros((self.n_hidden1, 1))
        b2 = np.zeros((self.n_hidden2, 1))
        b3 = np.zeros((n_output, 1))

        return W1, W2, W3, b1, b2, b3

    @staticmethod
    def _sigmoid(z):
        """Sigmoid activation function."""
        return expit(z)

    @staticmethod
    def _softmax(z):
        """Softmax activation function (for cross-entropy)."""
        exp_values = np.exp(z - np.max(z, axis=0, keepdims=True))  # stability
        return exp_values / np.sum(exp_values, axis=0, keepdims=True)

    @staticmethod
    def _cross_entropy_loss(Y_enc, A4):
        """Cross-entropy loss function."""
        m = Y_enc.shape[1]
        cost = -np.sum(Y_enc * np.log(A4 + 1e-8)) / m  # add small epsilon for stability
        return cost

    def _cost(self, A4, Y_enc, W1, W2, W3):
        """Compute the cost function with regularization."""
        cost = self._cross_entropy_loss(Y_enc, A4)
        L2_term = self._L2_reg(self.l2_C, W1, W2, W3)
        return cost + L2_term

    def _L2_reg(self, lambda_, W1, W2, W3):
        """Compute L2 regularization cost."""
        return (lambda_ / 2.0) * (np.sum(W1[:, 1:] ** 2) + np.sum(W2[:, 1:] ** 2) + np.sum(W3[:, 1:] ** 2))

    def _feedforward(self, X, W1, W2, W3, b1, b2, b3):
        """Compute the feedforward step."""
        A1 = self._add_bias_unit(X)
        Z1 = W1 @ A1.T + b1
        A2 = self._sigmoid(Z1)
        A2 = self._add_bias_unit(A2.T)
        Z2 = W2 @ A2.T + b2
        A3 = self._sigmoid(Z2)
        A3 = self._add_bias_unit(A3.T)
        Z3 = W3 @ A3.T + b3
        A4 = self._softmax(Z3)
        return A1, Z1, A2, Z2, A3, Z3, A4

    def _add_bias_unit(self, X):
        """Add bias unit to the input data."""
        ones = np.ones((X.shape[0], 1))
        return np.hstack((ones, X))

    def _get_gradient(self, A1, A2, A3, A4, Z1, Z2, Z3, Y_enc, W1, W2, W3):
        """Compute gradient step using backpropagation."""
        V3 = A4 - Y_enc
        delta3 = W3[:, 1:].T @ V3
        delta3_t = delta3.T
        A3_nobias = A3[:, 1:]
        V2 = A3_nobias * (1 - A3_nobias) * delta3_t

        delta2 = W2[:, 1:].T @ V2.T
        delta2_t = delta2.T
        A2_nobias = A2[:, 1:]
        V1 = A2_nobias * (1 - A2_nobias) * delta2_t

        gradW3 = V3 @ A3
        gradW2 = V2.T @ A2
        gradW1 = V1.T @ A1

        gradb3 = np.sum(V3, axis=1).reshape((-1, 1))
        gradb2 = np.sum(V2.T, axis=1).reshape((-1, 1))
        gradb1 = np.sum(V1.T, axis=1).reshape((-1, 1))

        gradW1[:, 1:] += self.l2_C * W1[:, 1:]
        gradW2[:, 1:] += self.l2_C * W2[:, 1:]
        gradW3[:, 1:] += self.l2_C * W3[:, 1:]

        return gradW1, gradW2, gradW3, gradb1, gradb2, gradb3

    def fit(self, X, y):
        Y_enc = self._encode_labels(y)
        n_features = X.shape[1]
        n_output = Y_enc.shape[0]

        W1, W2, W3, b1, b2, b3 = self.initialize_weights(n_features, n_output)
        cost_history = []
        grad_mag1, grad_mag2, grad_mag3 = [], [], []

        for epoch in range(self.epochs):
            minibatch_cost = 0
            epoch_grad1, epoch_grad2, epoch_grad3 = [], [], []

            permutation = np.random.permutation(X.shape[0])
            X_shuffled = X[permutation]
            Y_enc_shuffled = Y_enc[:, permutation]

            for i in range(0, X.shape[0], self.minibatch_size):
                X_batch = X_shuffled[i:i+self.minibatch_size]
                Y_batch = Y_enc_shuffled[:, i:i+self.minibatch_size]

                A1, Z1, A2, Z2, A3, Z3, A4 = self._feedforward(X_batch, W1, W2, W3, b1, b2, b3)
                minibatch_cost = self._cost(A4, Y_batch, W1, W2, W3)

                gradW1, gradW2, gradW3, gradb1, gradb2, gradb3 = self._get_gradient(
                    A1, A2, A3, A4, Z1, Z2, Z3, Y_batch, W1, W2, W3
                )

                # Track gradient magnitudes per minibatch
                epoch_grad1.append(np.linalg.norm(gradW1))
                epoch_grad2.append(np.linalg.norm(gradW2))
                epoch_grad3.append(np.linalg.norm(gradW3))

                # Update parameters
                W1 -= self.eta * gradW1
                W2 -= self.eta * gradW2
                W3 -= self.eta * gradW3
                b1 -= self.eta * gradb1
                b2 -= self.eta * gradb2
                b3 -= self.eta * gradb3

            # Record average gradient magnitude for each epoch
            grad_mag1.append(np.mean(epoch_grad1))
            grad_mag2.append(np.mean(epoch_grad2))
            grad_mag3.append(np.mean(epoch_grad3))
            cost_history.append(minibatch_cost)

        # Store results
        self.W1, self.W2, self.W3 = W1, W2, W3
        self.b1, self.b2, self.b3 = b1, b2, b3
        self.cost_ = cost_history
        self.grad_mag1_, self.grad_mag2_, self.grad_mag3_ = grad_mag1, grad_mag2, grad_mag3
        return self

    def predict(self, X):
        _, _, _, _, _, _, A4 = self._feedforward(X, self.W1, self.W2, self.W3, self.b1, self.b2, self.b3)
        return np.argmax(A4, axis=0)

    def plot_gradient_flow(self):
        plt.figure(figsize=(10, 4))

        plt.subplot(1, 2, 1)
        plt.plot(range(len(self.cost_)), self.cost_, label="Loss", color='tab:blue')
        plt.xlabel("Epochs")
        plt.ylabel("Cost (Loss)")
        plt.title("Loss Function vs Epochs")
        plt.legend()

        plt.subplot(1, 2, 2)
        epochs = range(len(self.grad_mag1_))
        plt.plot(epochs, self.grad_mag1_, label='W1', color='tab:blue')
        plt.plot(epochs, self.grad_mag2_, label='W2', color='tab:orange')
        plt.plot(epochs, self.grad_mag3_, label='W3', color='tab:green')
        plt.xlabel("Epochs")
        plt.ylabel("Average Gradient Magnitudes")
        plt.title("Gradient Magnitudes During Training")
        plt.legend()

        plt.tight_layout()
        plt.show()


def train_evaluate_model(X_train, y_train, X_test, y_test, n_hidden1, n_hidden2, **params):
    model = ThreeLayerPerceptron(n_hidden1=n_hidden1, n_hidden2=n_hidden2, epochs=100, eta=0.001, minibatch_size=64)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"Training Accuracy: {train_acc}")
    print(f"Test Accuracy: {test_acc}")

    model.plot_gradient_flow()

    return model

In [ ]:
params = {'epochs': 500,
          'eta': 0.001,
          'minibatch_size': 64,
          'C': 0.01,
          'random_state': 42,
}

print("Training and evaluating Three-Layer Perceptron with 30 and 20 hidden units:")
hidden_3 = [30,20]
three_layer_model = train_evaluate_model(X_train_onehot_norm, y_train_balanced, X_test_onehot_norm, y_test, hidden_3[0], hidden_3[1], **params)


Based on the "Gradient Magnitudes During Training" graph above, the average gradient magnitudes differ greatly for each layer of the perceptron. For the first layer of the perceptron, the magnitudes start relatively low and then increase tremendously as the number of epochs also increases. This means that in the first layer, the model makes very large changes when updating the weights, which could potentially raise issues and result in exploding gradients. For the second layer, there is little to no increase in the gradient magnitudes since they appear to stay at around 7 for all of the epochs. The third layer had almost the exact opposite effect on the gradient magnitudes compared to the first layer. For this layer, the gradient magnitudes started off moderate to high and decreased sharply as the epochs increased before evening out at around 2 or 3 for the gradient magnitudes. Additionally, the loss for the three-layer perceptron was very similar to the two-layer perceptron, seeing as how both models started with a high cost at zero epochs and then dropped dramatically before leveling off at around 0 and 0.1.

3.2 Repeat the previous step, adding support for a fourth layer.

In [ ]:
class FourLayerPerceptron(ThreeLayerPerceptron):
    def __init__(self, n_hidden1=30, n_hidden2=20, n_hidden3=10, C=0.0, epochs=500, eta=0.001, minibatch_size=64, random_state=None):
            np.random.seed(random_state)
            self.n_hidden1 = n_hidden1
            self.n_hidden2 = n_hidden2
            self.n_hidden3 = n_hidden3
            self.l2_C = C
            self.epochs = epochs
            self.eta = eta
            self.minibatch_size = minibatch_size

    def initialize_weights(self, n_features, n_output):
        """Initialize weights using Glorot initialization."""
        limit_1 = np.sqrt(6 / (n_features + self.n_hidden1))
        limit_2 = np.sqrt(6 / (self.n_hidden1 + self.n_hidden2))
        limit_3 = np.sqrt(6 / (self.n_hidden2 + self.n_hidden3))
        limit_4 = np.sqrt(6 / (self.n_hidden3 + n_output))

        W1 = np.random.uniform(-limit_1, limit_1, (self.n_hidden1, n_features + 1))
        W2 = np.random.uniform(-limit_2, limit_2, (self.n_hidden2, self.n_hidden1 + 1))
        W3 = np.random.uniform(-limit_3, limit_3, (self.n_hidden3, self.n_hidden2 + 1))
        W4 = np.random.uniform(-limit_4, limit_4, (n_output, self.n_hidden3 + 1))

        b1 = np.zeros((self.n_hidden1, 1))
        b2 = np.zeros((self.n_hidden2, 1))
        b3 = np.zeros((self.n_hidden3, 1))
        b4 = np.zeros((n_output, 1))

        return W1, W2, W3, W4, b1, b2, b3, b4

    @staticmethod
    def _cross_entropy_loss(Y_enc, A5):
        """Cross-entropy loss function."""
        m = Y_enc.shape[1]
        cost = -np.sum(Y_enc * np.log(A5 + 1e-8)) / m
        return cost

    def _cost(self, A5, Y_enc, W1, W2, W3, W4):
        """Compute the cost function with regularization."""
        cost = self._cross_entropy_loss(Y_enc, A5)
        L2_term = self._L2_reg(self.l2_C, W1, W2, W3, W4)
        return cost + L2_term

    def _L2_reg(self, lambda_, W1, W2, W3, W4):
        """Compute L2 regularization cost."""
        return (lambda_ / 2.0) * (np.sum(W1[:, 1:] ** 2) + np.sum(W2[:, 1:] ** 2) + np.sum(W3[:, 1:] ** 2)
                                  + np.sum(W4[:, 1:] ** 2))

    def _feedforward(self, X, W1, W2, W3, W4, b1, b2, b3, b4):
        """Compute the feedforward step."""
        A1 = self._add_bias_unit(X)
        Z1 = W1 @ A1.T + b1
        A2 = self._sigmoid(Z1)
        A2 = self._add_bias_unit(A2.T)
        Z2 = W2 @ A2.T + b2
        A3 = self._sigmoid(Z2)
        A3 = self._add_bias_unit(A3.T)
        Z3 = W3 @ A3.T + b3
        A4 = self._sigmoid(Z3)
        A4 = self._add_bias_unit(A4.T)
        Z4 = W4 @ A4.T + b4
        A5 = self._softmax(Z4)
        return A1, Z1, A2, Z2, A3, Z3, A4, Z4, A5

    def _get_gradient(self, A1, A2, A3, A4, A5, Z1, Z2, Z3, Z4, Y_enc, W1, W2, W3, W4):
        """Compute gradient step using backpropagation."""
        V4 = A5 - Y_enc
        delta4 = W4[:, 1:].T @ V4
        delta4_t = delta4.T
        A4_nobias = A4[:, 1:]
        V3 = A4_nobias * (1 - A4_nobias) * delta4_t

        delta3 = W3[:, 1:].T @ V3.T
        delta3_t = delta3.T
        A3_nobias = A3[:, 1:]
        V2 = A3_nobias * (1 - A3_nobias) * delta3_t

        delta2 = W2[:, 1:].T @ V2.T
        delta2_t = delta2.T
        A2_nobias = A2[:, 1:]
        V1 = A2_nobias * (1 - A2_nobias) * delta2_t

        gradW4 = V4 @ A4
        gradW3 = V3.T @ A3
        gradW2 = V2.T @ A2
        gradW1 = V1.T @ A1

        gradb4 = np.sum(V4, axis=1).reshape((-1, 1))
        gradb3 = np.sum(V3.T, axis=1).reshape((-1, 1))
        gradb2 = np.sum(V2.T, axis=1).reshape((-1, 1))
        gradb1 = np.sum(V1.T, axis=1).reshape((-1, 1))

        gradW1[:, 1:] += self.l2_C * W1[:, 1:]
        gradW2[:, 1:] += self.l2_C * W2[:, 1:]
        gradW3[:, 1:] += self.l2_C * W3[:, 1:]
        gradW4[:, 1:] += self.l2_C * W4[:, 1:]

        return gradW1, gradW2, gradW3, gradW4, gradb1, gradb2, gradb3, gradb4

    def fit(self, X, y):
        Y_enc = self._encode_labels(y)
        n_features = X.shape[1]
        n_output = Y_enc.shape[0]

        W1, W2, W3, W4, b1, b2, b3, b4 = self.initialize_weights(n_features, n_output)
        cost_history = []
        grad_mag1, grad_mag2, grad_mag3, grad_mag4 = [], [], [], []

        for epoch in range(self.epochs):
            minibatch_cost = 0
            epoch_grad1, epoch_grad2, epoch_grad3, epoch_grad4 = [], [], [], []

            permutation = np.random.permutation(X.shape[0])
            X_shuffled = X[permutation]
            Y_enc_shuffled = Y_enc[:, permutation]

            for i in range(0, X.shape[0], self.minibatch_size):
                X_batch = X_shuffled[i:i+self.minibatch_size]
                Y_batch = Y_enc_shuffled[:, i:i+self.minibatch_size]

                A1, Z1, A2, Z2, A3, Z3, A4, Z4, A5 = self._feedforward(X_batch, W1, W2, W3, W4, b1, b2, b3, b4)
                minibatch_cost = self._cost(A5, Y_batch, W1, W2, W3, W4)

                gradW1, gradW2, gradW3, gradW4, gradb1, gradb2, gradb3, gradb4 = self._get_gradient(
                    A1, A2, A3, A4, A5, Z1, Z2, Z3, Z4, Y_batch, W1, W2, W3, W4
                )

                # Track gradient magnitudes per minibatch
                epoch_grad1.append(np.linalg.norm(gradW1))
                epoch_grad2.append(np.linalg.norm(gradW2))
                epoch_grad3.append(np.linalg.norm(gradW3))
                epoch_grad4.append(np.linalg.norm(gradW4))

                # Update parameters
                W1 -= self.eta * gradW1
                W2 -= self.eta * gradW2
                W3 -= self.eta * gradW3
                W4 -= self.eta * gradW4
                b1 -= self.eta * gradb1
                b2 -= self.eta * gradb2
                b3 -= self.eta * gradb3
                b4 -= self.eta * gradb4

            # Record average gradient magnitude for each epoch
            grad_mag1.append(np.mean(epoch_grad1))
            grad_mag2.append(np.mean(epoch_grad2))
            grad_mag3.append(np.mean(epoch_grad3))
            grad_mag4.append(np.mean(epoch_grad4))
            cost_history.append(minibatch_cost)

        # Store results
        self.W1, self.W2, self.W3, self.W4 = W1, W2, W3, W4
        self.b1, self.b2, self.b3, self.b4 = b1, b2, b3, b4
        self.cost_ = cost_history
        self.grad_mag1_, self.grad_mag2_, self.grad_mag3_, self.grad_mag4_ = grad_mag1, grad_mag2, grad_mag3, grad_mag4
        return self

    def predict(self, X):
        _, _, _, _, _, _, _, _, A5 = self._feedforward(X, self.W1, self.W2, self.W3, self.W4, self.b1,
                                                 self.b2, self.b3, self.b4)
        return np.argmax(A5, axis=0)

    def plot_gradient_flow(self):
        plt.figure(figsize=(10, 4))

        plt.subplot(1, 2, 1)
        plt.plot(range(len(self.cost_)), self.cost_, label="Loss", color='tab:blue')
        plt.xlabel("Epochs")
        plt.ylabel("Cost (Loss)")
        plt.title("Loss Function vs Epochs")
        plt.legend()

        plt.subplot(1, 2, 2)
        epochs = range(len(self.grad_mag1_))
        plt.plot(epochs, self.grad_mag1_, label='W1', color='tab:blue')
        plt.plot(epochs, self.grad_mag2_, label='W2', color='tab:orange')
        plt.plot(epochs, self.grad_mag3_, label='W3', color='tab:green')
        plt.plot(epochs, self.grad_mag4_, label='W4', color='tab:red')
        plt.xlabel("Epochs")
        plt.ylabel("Average Gradient Magnitudes")
        plt.title("Gradient Magnitudes During Training")
        plt.legend()

        plt.tight_layout()
        plt.show()


def train_evaluate_model(X_train, y_train, X_test, y_test, n_hidden1, n_hidden2, n_hidden3, **params):
    model = FourLayerPerceptron(n_hidden1=n_hidden1, n_hidden2=n_hidden2, n_hidden3=n_hidden3,
                                 epochs=100, eta=0.001, minibatch_size=64)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_acc = accuracy_score(y_train, y_train_pred)
    test_acc = accuracy_score(y_test, y_test_pred)

    print(f"Training Accuracy: {train_acc}")
    print(f"Test Accuracy: {test_acc}")

    model.plot_gradient_flow()

    return model

In [ ]:
print("Training and evaluating a Four-Layer Perceptron with 30, 20, and 10 hidden units:")
hidden_layers = [30,20,10]
four_layer_model = train_evaluate_model(X_train_onehot_norm, y_train_balanced, X_test_onehot_norm,
                                         y_test, hidden_layers[0], hidden_layers[1], hidden_layers[2], **params)

For the four-layer perceptron model, the first layer performed quite similarly to that of the three-layer perceptron model, since the average gradient magnitudes of both started off small and increased greatly as the number of epochs increased. However, unlike with the three-layer perceptron, the four-layer perceptron's second layer saw a noticeable increase in the average gradient magnitudes from 0 to about 40 epochs before leveling off at a magnitude of roughly 15. Furthermore, the third layer's magnitudes did not see a large drop like with the three-layer perceptron, but instead saw a small increase from about 0 to 10 epochs before evening out and remaining at an average gradient magnitude of about 7 for the rest of the epochs. For the four-layer perceptron model, it was the fourth layer instead of the third layer that saw a large decrease in the average gradient magnitudes before converging to a magnitude of roughly 2. This is due to the cross entropy loss function using the deepest layer of the perceptron model to calculate the difference between the model's prediction and the ground truth.

3.3 Repeat the previous step, adding support for a fifth layer.

In [ ]:
class FiveLayerPerceptron(FourLayerPerceptron):
    def __init__(self, n_hidden1=30, n_hidden2=20, n_hidden3=10, n_hidden4=5,
                 C=0.0, epochs=500, eta=0.001, minibatch_size=64, random_state=None):
        np.random.seed(random_state)
        self.n_hidden1 = n_hidden1
        self.n_hidden2 = n_hidden2
        self.n_hidden3 = n_hidden3
        self.n_hidden4 = n_hidden4
        self.l2_C = C
        self.epochs = epochs
        self.eta = eta
        self.minibatch_size = minibatch_size

    def initialize_weights(self, n_features, n_output):
        # glorot initialization
        limit_1 = np.sqrt(6 / (n_features + self.n_hidden1))
        limit_2 = np.sqrt(6 / (self.n_hidden1 + self.n_hidden2))
        limit_3 = np.sqrt(6 / (self.n_hidden2 + self.n_hidden3))
        limit_4 = np.sqrt(6 / (self.n_hidden3 + self.n_hidden4))
        limit_5 = np.sqrt(6 / (self.n_hidden4 + n_output))

        W1 = np.random.uniform(-limit_1, limit_1, (self.n_hidden1, n_features + 1))
        W2 = np.random.uniform(-limit_2, limit_2, (self.n_hidden2, self.n_hidden1 + 1))
        W3 = np.random.uniform(-limit_3, limit_3, (self.n_hidden3, self.n_hidden2 + 1))
        W4 = np.random.uniform(-limit_4, limit_4, (self.n_hidden4, self.n_hidden3 + 1))
        W5 = np.random.uniform(-limit_5, limit_5, (n_output, self.n_hidden4 + 1))

        b1 = np.zeros((self.n_hidden1, 1))
        b2 = np.zeros((self.n_hidden2, 1))
        b3 = np.zeros((self.n_hidden3, 1))
        b4 = np.zeros((self.n_hidden4, 1))
        b5 = np.zeros((n_output, 1))

        return W1, W2, W3, W4, W5, b1, b2, b3, b4, b5

    @staticmethod
    def _cross_entropy_loss(Y_enc, A6):
        """Cross-entropy loss function."""
        m = Y_enc.shape[1]
        cost = -np.sum(Y_enc * np.log(A6 + 1e-8)) / m
        return cost

    def _cost(self, A6, Y_enc, W1, W2, W3, W4, W5):
        """Compute the cost function with regularization."""
        cost = self._cross_entropy_loss(Y_enc, A6)
        L2_term = self._L2_reg(self.l2_C, W1, W2, W3, W4, W5)
        return cost + L2_term

    def _L2_reg(self, lambda_, W1, W2, W3, W4, W5):
        """Compute L2 regularization cost."""
        return (lambda_ / 2.0) * (np.sum(W1[:, 1:] ** 2) + np.sum(W2[:, 1:] ** 2) + np.sum(W3[:, 1:] ** 2)
                                  + np.sum(W4[:, 1:] ** 2) + np.sum(W5[:, 1:] ** 2))

    def _feedforward(self, X, W1, W2, W3, W4, W5, b1, b2, b3, b4, b5):
        A1 = self._add_bias_unit(X)
        Z1 = W1 @ A1.T + b1
        A2 = self._sigmoid(Z1)
        A2 = self._add_bias_unit(A2.T)
        Z2 = W2 @ A2.T + b2
        A3 = self._sigmoid(Z2)
        A3 = self._add_bias_unit(A3.T)
        Z3 = W3 @ A3.T + b3
        A4 = self._sigmoid(Z3)
        A4 = self._add_bias_unit(A4.T)
        Z4 = W4 @ A4.T + b4
        A5 = self._sigmoid(Z4)
        A5 = self._add_bias_unit(A5.T)
        Z5 = W5 @ A5.T + b5
        A6 = self._softmax(Z5)
        return A1, Z1, A2, Z2, A3, Z3, A4, Z4, A5, Z5, A6

    def _get_gradient(self, A1, A2, A3, A4, A5, A6, Z1, Z2, Z3, Z4, Z5, Y_enc, W1, W2, W3, W4, W5):
        # backprop
        V5 = A6 - Y_enc
        delta5 = W5[:, 1:].T @ V5
        delta5_t = delta5.T
        A5_nobias = A5[:, 1:]
        V4 = A5_nobias * (1 - A5_nobias) * delta5_t

        delta4 = W4[:, 1:].T @ V4.T
        delta4_t = delta4.T
        A4_nobias = A4[:, 1:]
        V3 = A4_nobias * (1 - A4_nobias) * delta4_t

        delta3 = W3[:, 1:].T @ V3.T
        delta3_t = delta3.T
        A3_nobias = A3[:, 1:]
        V2 = A3_nobias * (1 - A3_nobias) * delta3_t

        delta2 = W2[:, 1:].T @ V2.T
        delta2_t = delta2.T
        A2_nobias = A2[:, 1:]
        V1 = A2_nobias * (1 - A2_nobias) * delta2_t
        # gradients
        gradW5 = V5 @ A5
        gradW4 = V4.T @ A4
        gradW3 = V3.T @ A3
        gradW2 = V2.T @ A2
        gradW1 = V1.T @ A1

        gradb5 = np.sum(V5, axis=1).reshape((-1, 1))
        gradb4 = np.sum(V4.T, axis=1).reshape((-1, 1))
        gradb3 = np.sum(V3.T, axis=1).reshape((-1, 1))
        gradb2 = np.sum(V2.T, axis=1).reshape((-1, 1))
        gradb1 = np.sum(V1.T, axis=1).reshape((-1, 1))

        gradW1[:, 1:] += self.l2_C * W1[:, 1:]
        gradW2[:, 1:] += self.l2_C * W2[:, 1:]
        gradW3[:, 1:] += self.l2_C * W3[:, 1:]
        gradW4[:, 1:] += self.l2_C * W4[:, 1:]
        gradW5[:, 1:] += self.l2_C * W5[:, 1:]

        return gradW1, gradW2, gradW3, gradW4, gradW5, gradb1, gradb2, gradb3, gradb4, gradb5

    def fit(self, X, y):
        Y_enc = self._encode_labels(y)
        n_features = X.shape[1]
        n_output = Y_enc.shape[0]

        W1, W2, W3, W4, W5, b1, b2, b3, b4, b5 = self.initialize_weights(n_features, n_output)
        cost_history = []
        grad_mag1, grad_mag2, grad_mag3, grad_mag4, grad_mag5 = [], [], [], [], []

        for epoch in range(self.epochs):
            minibatch_cost = 0
            g1, g2, g3, g4, g5 = [], [], [], [], []

            permutation = np.random.permutation(X.shape[0])
            X_shuffled = X[permutation]
            Y_enc_shuffled = Y_enc[:, permutation]

            for i in range(0, X.shape[0], self.minibatch_size):
                X_batch = X_shuffled[i:i+self.minibatch_size]
                Y_batch = Y_enc_shuffled[:, i:i+self.minibatch_size]

                A1, Z1, A2, Z2, A3, Z3, A4, Z4, A5, Z5, A6 = self._feedforward(
                    X_batch, W1, W2, W3, W4, W5, b1, b2, b3, b4, b5
                )
                minibatch_cost = self._cost(A6, Y_batch, W1, W2, W3, W4, W5)
                grads = self._get_gradient(A1, A2, A3, A4, A5, A6, Z1, Z2, Z3, Z4, Z5,
                                           Y_batch, W1, W2, W3, W4, W5)

                gradW1, gradW2, gradW3, gradW4, gradW5, gradb1, gradb2, gradb3, gradb4, gradb5 = grads

                # track gradient
                g1.append(np.linalg.norm(gradW1))
                g2.append(np.linalg.norm(gradW2))
                g3.append(np.linalg.norm(gradW3))
                g4.append(np.linalg.norm(gradW4))
                g5.append(np.linalg.norm(gradW5))

                # update weights
                W1 -= self.eta * gradW1
                W2 -= self.eta * gradW2
                W3 -= self.eta * gradW3
                W4 -= self.eta * gradW4
                W5 -= self.eta * gradW5
                b1 -= self.eta * gradb1
                b2 -= self.eta * gradb2
                b3 -= self.eta * gradb3
                b4 -= self.eta * gradb4
                b5 -= self.eta * gradb5

            grad_mag1.append(np.mean(g1))
            grad_mag2.append(np.mean(g2))
            grad_mag3.append(np.mean(g3))
            grad_mag4.append(np.mean(g4))
            grad_mag5.append(np.mean(g5))
            cost_history.append(minibatch_cost)

        # store everything
        self.W1, self.W2, self.W3, self.W4, self.W5 = W1, W2, W3, W4, W5
        self.b1, self.b2, self.b3, self.b4, self.b5 = b1, b2, b3, b4, b5
        self.cost_ = cost_history
        self.grad_mag1_, self.grad_mag2_, self.grad_mag3_, self.grad_mag4_, self.grad_mag5_ = (
            grad_mag1, grad_mag2, grad_mag3, grad_mag4, grad_mag5
        )
        return self

    def predict(self, X):
        _, _, _, _, _, _, _, _, _, _, A6 = self._feedforward(
            X, self.W1, self.W2, self.W3, self.W4, self.W5,
            self.b1, self.b2, self.b3, self.b4, self.b5
        )
        return np.argmax(A6, axis=0)

    def plot_gradient_flow(self):
        plt.figure(figsize=(10, 4))

        plt.subplot(1, 2, 1)
        plt.plot(range(len(self.cost_)), self.cost_, label="loss", color='tab:blue')
        plt.xlabel("epochs")
        plt.ylabel("cost (loss)")
        plt.title("loss function vs epochs")
        plt.legend()

        plt.subplot(1, 2, 2)
        epochs = range(len(self.grad_mag1_))
        plt.plot(epochs, self.grad_mag1_, label='W1')
        plt.plot(epochs, self.grad_mag2_, label='W2')
        plt.plot(epochs, self.grad_mag3_, label='W3')
        plt.plot(epochs, self.grad_mag4_, label='W4')
        plt.plot(epochs, self.grad_mag5_, label='W5')
        plt.xlabel("epochs")
        plt.ylabel("average gradient magnitudes")
        plt.title("gradient magnitudes during training")
        plt.legend()
        plt.tight_layout()
        plt.show()

def train_evaluate_model(X_train, y_train, X_test, y_test, n_hidden1, n_hidden2, n_hidden3, n_hidden4, **params):
    model = FiveLayerPerceptron(n_hidden1=n_hidden1, n_hidden2=n_hidden2,
                                n_hidden3=n_hidden3, n_hidden4=n_hidden4,
                                epochs=100, eta=0.001, minibatch_size=64)
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
    print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

    model.plot_gradient_flow()
    return model

In [ ]:
print("Training and evaluating a Five-Layer Perceptron with 30, 20, 10 and 5 hidden units:")
hidden_layers = [30, 20, 10, 5]
five_layer_model = train_evaluate_model(
    X_train_onehot_norm, y_train_balanced,
    X_test_onehot_norm, y_test,
    hidden_layers[0], hidden_layers[1], hidden_layers[2], hidden_layers[3], **params
)

For the five layer perceptron model, the first layer showed a really steep increase in the average gradient magnitudes within the first few epochs before leveling off at around 40. The second and third layers increased at smaller magnitudes, and reached values at around 10-20 at around 20 epochs and above. The fourth layer started off with higher gradients but decreased and flattened around a magnitude of 5. The fifth layer saw a drop early on before arriving at the lowest magnitude, around 3. The graph shows that as depth increases, the gradient magnitudes become smaller.

3.4 Implement an adaptive learning technique that was discussed in lecture and use it on the five layer network (choose either RMSProp or AdaDelta). Discuss which adaptive method you chose. Compare the performance of your five layer model with and without the adaptive learning strategy. **Do not use AdaM for the adaptive learning technique as it is part of the exceptional work.**

In [ ]:
class RMSPropFLP(FiveLayerPerceptron):
    def __init__(self, alpha=0.1, **kwds):
        super().__init__(**kwds)
        self.alpha = alpha

    def _rmsprop_update(self, gradW1, gradW2, gradW3, gradW4, gradW5):
        if not hasattr(self,'_V1_prev'):
            self._V1_prev = np.zeros_like(gradW1)
            self._V2_prev = np.zeros_like(gradW2)
            self._V3_prev = np.zeros_like(gradW3)
            self._V4_prev = np.zeros_like(gradW4)
            self._V5_prev = np.zeros_like(gradW5)

        G1 = gradW1*gradW1
        G2 = gradW2*gradW2
        G3 = gradW3*gradW3
        G4 = gradW4*gradW4
        G5 = gradW5*gradW5

        V1 = self.alpha*self._V1_prev + (1-self.alpha)*G1 + 1e-7
        V2 = self.alpha*self._V2_prev + (1-self.alpha)*G2 + 1e-7
        V3 = self.alpha*self._V3_prev + (1-self.alpha)*G3 + 1e-7
        V4 = self.alpha*self._V4_prev + (1-self.alpha)*G4 + 1e-7
        V5 = self.alpha*self._V5_prev + (1-self.alpha)*G5 + 1e-7

        gradW1 = gradW1/np.sqrt(V1)
        gradW2 = gradW2/np.sqrt(V2)
        gradW3 = gradW3/np.sqrt(V3)
        gradW4 = gradW4/np.sqrt(V4)
        gradW5 = gradW5/np.sqrt(V5)

        # save previous updates
        self._V1_prev, self._V2_prev, self._V3_prev, self._V4_prev, self._V5_prev = V1, V2, V3, V4, V5

        return gradW1, gradW2, gradW3, gradW4, gradW5

    def fit(self, X, y):
        Y_enc = self._encode_labels(y)
        n_features = X.shape[1]
        n_output = Y_enc.shape[0]

        W1, W2, W3, W4, W5, b1, b2, b3, b4, b5 = self.initialize_weights(n_features, n_output)
        cost_history = []
        grad_mag1, grad_mag2, grad_mag3, grad_mag4, grad_mag5 = [], [], [], [], []

        for epoch in range(self.epochs):
            minibatch_cost = 0
            g1, g2, g3, g4, g5 = [], [], [], [], []

            permutation = np.random.permutation(X.shape[0])
            X_shuffled = X[permutation]
            Y_enc_shuffled = Y_enc[:, permutation]

            for i in range(0, X.shape[0], self.minibatch_size):
                X_batch = X_shuffled[i:i+self.minibatch_size]
                Y_batch = Y_enc_shuffled[:, i:i+self.minibatch_size]

                A1, Z1, A2, Z2, A3, Z3, A4, Z4, A5, Z5, A6 = self._feedforward(
                    X_batch, W1, W2, W3, W4, W5, b1, b2, b3, b4, b5
                )
                minibatch_cost = self._cost(A6, Y_batch, W1, W2, W3, W4, W5)
                grads = self._get_gradient(A1, A2, A3, A4, A5, A6, Z1, Z2, Z3, Z4, Z5,
                                           Y_batch, W1, W2, W3, W4, W5)

                gradW1, gradW2, gradW3, gradW4, gradW5, gradb1, gradb2, gradb3, gradb4, gradb5 = grads
                # updates weights using rmsprop
                gradW1, gradW2, gradW3, gradW4, gradW5 = self._rmsprop_update(gradW1, gradW2, gradW3, gradW4, gradW5)

                # track gradient
                g1.append(np.linalg.norm(gradW1))
                g2.append(np.linalg.norm(gradW2))
                g3.append(np.linalg.norm(gradW3))
                g4.append(np.linalg.norm(gradW4))
                g5.append(np.linalg.norm(gradW5))

                # update weights
                W1 -= self.eta * gradW1
                W2 -= self.eta * gradW2
                W3 -= self.eta * gradW3
                W4 -= self.eta * gradW4
                W5 -= self.eta * gradW5
                b1 -= self.eta * gradb1
                b2 -= self.eta * gradb2
                b3 -= self.eta * gradb3
                b4 -= self.eta * gradb4
                b5 -= self.eta * gradb5

            grad_mag1.append(np.mean(g1))
            grad_mag2.append(np.mean(g2))
            grad_mag3.append(np.mean(g3))
            grad_mag4.append(np.mean(g4))
            grad_mag5.append(np.mean(g5))
            cost_history.append(minibatch_cost)

        # store everything
        self.W1, self.W2, self.W3, self.W4, self.W5 = W1, W2, W3, W4, W5
        self.b1, self.b2, self.b3, self.b4, self.b5 = b1, b2, b3, b4, b5
        self.cost_ = cost_history
        self.grad_mag1_, self.grad_mag2_, self.grad_mag3_, self.grad_mag4_, self.grad_mag5_ = (
            grad_mag1, grad_mag2, grad_mag3, grad_mag4, grad_mag5
        )
        return self

def train_evaluate_model(X_train, y_train, X_test, y_test, n_hidden1, n_hidden2, n_hidden3, n_hidden4, **params):
    model = RMSPropFLP(n_hidden1=n_hidden1, n_hidden2=n_hidden2,
                                n_hidden3=n_hidden3, n_hidden4=n_hidden4,
                                alpha=0.1,epochs=100, eta=0.001, minibatch_size=64)
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
    print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

    model.plot_gradient_flow()
    return model

In [ ]:
print("Implementation of RMSProp with the Five-Layer Perceptron:")
hidden_layers = [30, 20, 10, 5]
rms = train_evaluate_model(X_train_onehot_norm, y_train_balanced, X_test_onehot_norm,
                                         y_test, hidden_layers[0], hidden_layers[1],
                           hidden_layers[2], hidden_layers[3], **params)


The adaptive method we chose was RMSProp because it gives us a way to mitigate the effects of vanishing gradients. As the model goes deeper into its layers, the average gradient magnitudes decrease and eventually result in vanishing gradients since models reach a point where the weights are remaining essentially the same for all epochs. However, with RMSProp, the learning rate changes with the magnitude of the gradients to ensure stability despite differences in the magnitudes, thereby reducing the effect of vanishing gradients.

The standard five layer perceptron had a 0.9875 training accuracy and a 0.9827 test accuracy. On the other hand, RMSProp five layer perceptron had a 0.9889 training accuracy and a 0.9838 test accuracy. Both models did well, but RMSProp showed slight improvement in both the training and the test accuracy. Though the improvement is small, it's still meaningful since the models are very close to perfect classification. Looking at the magnitude plots, the standard model depicts the gradient magnitudes increase quickly but then flattens for each layer. On the otherhand, for RMSProp, the gradients are lower but they're more balanced across each layers.

### **4. Exceptional Work**

Implement adaptive momentum (AdaM) in the five layer neural network and quantify the performance compared to other methods.  